In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'14rjljlj'

In [3]:
wandb.init(id='3jf8wk1i', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 55.6 s, sys: 2.58 s, total: 58.1 s
Wall time: 1min


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


In [9]:
for n, p in m.named_parameters():
    """if 'fcs_2' in n:
        p.requires_grad = True
    else:
        p.requires_grad = False"""
        
    print(n, p.requires_grad)

hidden_init True
cell_init True
backbone.conv_stem.weight True
backbone.bn1.weight True
backbone.bn1.bias True
backbone.blocks.0.0.conv_dw.weight True
backbone.blocks.0.0.bn1.weight True
backbone.blocks.0.0.bn1.bias True
backbone.blocks.0.0.se.conv_reduce.weight True
backbone.blocks.0.0.se.conv_reduce.bias True
backbone.blocks.0.0.se.conv_expand.weight True
backbone.blocks.0.0.se.conv_expand.bias True
backbone.blocks.0.0.conv_pw.weight True
backbone.blocks.0.0.bn2.weight True
backbone.blocks.0.0.bn2.bias True
backbone.blocks.0.1.conv_dw.weight True
backbone.blocks.0.1.bn1.weight True
backbone.blocks.0.1.bn1.bias True
backbone.blocks.0.1.se.conv_reduce.weight True
backbone.blocks.0.1.se.conv_reduce.bias True
backbone.blocks.0.1.se.conv_expand.weight True
backbone.blocks.0.1.se.conv_expand.bias True
backbone.blocks.0.1.conv_pw.weight True
backbone.blocks.0.1.bn2.weight True
backbone.blocks.0.1.bn2.bias True
backbone.blocks.1.0.conv_pw.weight True
backbone.blocks.1.0.bn1.weight True
backb

In [10]:
model_stem = "8.18"
START_E = 64

In [11]:
n_epochs = 1000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = 6e-4 if e < 46 else 3e-4 if e < 55 else 1e-4
    #opt.param_groups[0]['lr'] = 1e-4 #6e-4 if e < 1 else 3e-4 if e < 2 else 1e-4
    #opt.param_groups[0]['lr'] = 1e-4
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 64

{'trn_control_loss': 0.00873174, 'trn_headings_loss': 0.00039763, 'trn_curvatures_loss': 0.00163812, 'consistency losses/trn_steer_cost': 0.00378384, 'consistency losses/trn_te_loss': 17.78492679, 'aux losses/trn_pitch_loss': 0.10069251, 'aux losses/trn_yaw_loss': 0.15883446, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.1796875, 'consistency losses/trn_torque_loss': 532.04628754, 'consistency losses/trn_torque_delta_loss': 6.88113113, 'logistical/obs_consumed_per_second': 235.55905512, 'logistical/obs_generated_per_second': 97.74015748, 'logistical/data_consumption_ratio': 2.48192597, 'logistical/manual_train_pause': 0.02354331, 'logistical/max_param': 16.19180107, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 233.953125, 'logistical/obs_generated_per_second': 99.9921875, 'logistical/data_consumption_ratio': 2.34361556, 'logistical/manual_train_pause': 0.00023438, 'trn_control_loss': 0.00990683, 'trn_headings_loss': 0.00045236, 'trn_curvatures_l

{'logistical/obs_consumed_per_second': 223.578125, 'logistical/obs_generated_per_second': 103.8984375, 'logistical/data_consumption_ratio': 2.15535797, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01301128, 'trn_headings_loss': 0.00052293, 'trn_curvatures_loss': 0.00211935, 'consistency losses/trn_steer_cost': 0.00408545, 'consistency losses/trn_te_loss': 18.28395197, 'aux losses/trn_pitch_loss': 0.1001668, 'aux losses/trn_yaw_loss': 0.20403814, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.296875, 'consistency losses/trn_torque_loss': 589.4606709, 'consistency losses/trn_torque_delta_loss': 7.27189309, 'logistical/max_param': 16.18839455, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 234.625, 'logistical/obs_generated_per_second': 109.09375, 'logistical/data_consumption_ratio': 2.1508625, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00948249, 'trn_headings_loss': 0.00036044, 'trn_curvatures_loss': 0.00139405, 'consistency los

{'logistical/obs_consumed_per_second': 230.78125, 'logistical/obs_generated_per_second': 96.7578125, 'logistical/data_consumption_ratio': 2.38620429, 'logistical/manual_train_pause': 0.00148438, 'trn_control_loss': 0.00831062, 'trn_headings_loss': 0.00032624, 'trn_curvatures_loss': 0.00115609, 'consistency losses/trn_steer_cost': 0.00247076, 'consistency losses/trn_te_loss': 19.07455549, 'aux losses/trn_pitch_loss': 0.08837795, 'aux losses/trn_yaw_loss': 0.11374521, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.2421875, 'consistency losses/trn_torque_loss': 629.10323114, 'consistency losses/trn_torque_delta_loss': 6.42926301, 'logistical/max_param': 16.18722916, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 231.796875, 'logistical/obs_generated_per_second': 104.4765625, 'logistical/data_consumption_ratio': 2.22178977, 'logistical/manual_train_pause': 7.812e-05, 'trn_control_loss': 0.00642343, 'trn_headings_loss': 0.0002779, 'trn_curvatures_loss': 0.0010254

{'logistical/obs_consumed_per_second': 237.015625, 'logistical/obs_generated_per_second': 101.7890625, 'logistical/data_consumption_ratio': 2.33167671, 'logistical/manual_train_pause': 0.00023438, 'trn_control_loss': 0.00767219, 'trn_headings_loss': 0.00032887, 'trn_curvatures_loss': 0.00109006, 'consistency losses/trn_steer_cost': 0.0026694, 'consistency losses/trn_te_loss': 19.288459, 'aux losses/trn_pitch_loss': 0.08580554, 'aux losses/trn_yaw_loss': 0.0937711, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.1875, 'consistency losses/trn_torque_loss': 787.48752664, 'consistency losses/trn_torque_delta_loss': 5.70926723, 'logistical/max_param': 16.18362999, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 230.6171875, 'logistical/obs_generated_per_second': 100.734375, 'logistical/data_consumption_ratio': 2.28748651, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00572179, 'trn_headings_loss': 0.00025542, 'trn_curvatures_loss': 0.00098751, 'consis

{'logistical/obs_consumed_per_second': 229.015625, 'logistical/obs_generated_per_second': 101.84375, 'logistical/data_consumption_ratio': 2.25508553, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00931337, 'trn_headings_loss': 0.00036976, 'trn_curvatures_loss': 0.00131957, 'consistency losses/trn_steer_cost': 0.00334482, 'consistency losses/trn_te_loss': 18.1284281, 'aux losses/trn_pitch_loss': 0.07965887, 'aux losses/trn_yaw_loss': 0.13489389, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.2890625, 'consistency losses/trn_torque_loss': 661.83124927, 'consistency losses/trn_torque_delta_loss': 7.39676686, 'logistical/max_param': 16.18844986, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 235.453125, 'logistical/obs_generated_per_second': 108.109375, 'logistical/data_consumption_ratio': 2.18094428, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0105519, 'trn_headings_loss': 0.00041584, 'trn_curvatures_loss': 0.00133619, 'consistency

...

{'logistical/obs_consumed_per_second': 230.671875, 'logistical/obs_generated_per_second': 105.5859375, 'logistical/data_consumption_ratio': 2.18694842, 'logistical/manual_train_pause': 0.00015625, 'trn_control_loss': 0.00784901, 'trn_headings_loss': 0.00028153, 'trn_curvatures_loss': 0.00123261, 'consistency losses/trn_steer_cost': 0.0030895, 'consistency losses/trn_te_loss': 17.8293394, 'aux losses/trn_pitch_loss': 0.10269284, 'aux losses/trn_yaw_loss': 0.14472675, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.1953125, 'consistency losses/trn_torque_loss': 729.45321461, 'consistency losses/trn_torque_delta_loss': 5.00436511, 'logistical/max_param': 16.18406296, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 236.4296875, 'logistical/obs_generated_per_second': 90.4453125, 'logistical/data_consumption_ratio': 2.6160371, 'logistical/manual_train_pause': 0.00289062, 'trn_control_loss': 0.0063614, 'trn_headings_loss': 0.00028123, 'trn_curvatures_loss': 0.0011229

{'logistical/obs_consumed_per_second': 131.0, 'logistical/obs_generated_per_second': 102.0, 'logistical/data_consumption_ratio': 1.28911887, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 88

{'trn_control_loss': 0.00698876, 'trn_headings_loss': 0.00027269, 'trn_curvatures_loss': 0.00100309, 'consistency losses/trn_steer_cost': 0.00257922, 'consistency losses/trn_te_loss': 19.37902056, 'aux losses/trn_pitch_loss': 0.08156419, 'aux losses/trn_yaw_loss': 0.09703565, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.25, 'consistency losses/trn_torque_loss': 780.35914743, 'consistency losses/trn_torque_delta_loss': 5.55365421, 'logistical/obs_consumed_per_second': 238.13385827, 'logistical/obs_generated_per_second': 95.09448819, 'logistical/data_consumption_ratio': 2.50191292, 'logistical/manual_train_pause': 0.00173228, 'logistical/max_param': 16.18057632, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 235.9453125, 'logistical/obs_generated_per_second': 10

{'logistical/obs_consumed_per_second': 236.4375, 'logistical/obs_generated_per_second': 105.0234375, 'logistical/data_consumption_ratio': 2.25417932, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00805923, 'trn_headings_loss': 0.00033268, 'trn_curvatures_loss': 0.00140314, 'consistency losses/trn_steer_cost': 0.00339807, 'consistency losses/trn_te_loss': 17.62372751, 'aux losses/trn_pitch_loss': 0.06964064, 'aux losses/trn_yaw_loss': 0.13848782, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.109375, 'consistency losses/trn_torque_loss': 602.32214937, 'consistency losses/trn_torque_delta_loss': 4.86043417, 'logistical/max_param': 16.1848011, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 233.71875, 'logistical/obs_generated_per_second': 104.953125, 'logistical/data_consumption_ratio': 2.22805355, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00737011, 'trn_headings_loss': 0.0002462, 'trn_curvatures_loss': 0.000948, 'consistency los

{'logistical/obs_consumed_per_second': 231.2265625, 'logistical/obs_generated_per_second': 106.9609375, 'logistical/data_consumption_ratio': 2.17038651, 'logistical/manual_train_pause': 7.812e-05, 'trn_control_loss': 0.00827834, 'trn_headings_loss': 0.0002996, 'trn_curvatures_loss': 0.00130834, 'consistency losses/trn_steer_cost': 0.00285789, 'consistency losses/trn_te_loss': 18.76902273, 'aux losses/trn_pitch_loss': 0.09166098, 'aux losses/trn_yaw_loss': 0.10470843, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.203125, 'consistency losses/trn_torque_loss': 714.50223214, 'consistency losses/trn_torque_delta_loss': 7.5608098, 'logistical/max_param': 16.18796158, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 235.5, 'logistical/obs_generated_per_second': 108.484375, 'logistical/data_consumption_ratio': 2.17403525, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00994219, 'trn_headings_loss': 0.00031872, 'trn_curvatures_loss': 0.00125714, 'consiste

KeyboardInterrupt: 

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e63.torch")

In [12]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
